In [16]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from qutip_qip.circuit import QubitCircuit

import pennylane as qml


np.random.seed(42)

In [17]:
dataset = pd.read_csv("./Homework_Datasets/Pizza.csv")
dataset['brand'] = dataset['brand'].apply(lambda x: 0 if x == 'A' else 1)

X=dataset.iloc[: , 2:8].values
y=dataset.iloc[: ,0].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.33, random_state=42)

sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

print(X)

[[27.82 21.43 44.87  5.11  1.77  0.77]
 [28.49 21.26 43.89  5.34  1.79  1.02]
 [28.35 19.99 45.78  5.08  1.63  0.8 ]
 ...
 [44.55 11.01 16.03  2.43  0.64 25.98]
 [47.6  10.43 15.18  2.32  0.56 24.47]
 [46.84  9.91 15.5   2.27  0.57 25.48]]


In [18]:
n_qubits = 4

def normalize_data(vector):
    norm_vector = np.pi * (2*(vector - vector.min()) / (vector.max() - vector.min()) - 1) / 2
    return norm_vector

def encode_data(vector):
    norm_vector = normalize_data(vector)
    circuit = QubitCircuit(n_qubits)
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[0])
    circuit.add_gate("RX", targets=[1], arg_value=norm_vector[1])
    circuit.add_gate("CNOT", controls=[0], targets=[1])
    circuit.add_gate("CNOT", controls=[1], targets=[2])
    circuit.add_gate("CNOT", controls=[2], targets=[3])
    circuit.add_gate("RX", targets=[0], arg_value=norm_vector[3])
    circuit.add_gate("RX", targets=[1], arg_value=norm_vector[4])
    circuit.add_gate("RX", targets=[2], arg_value=norm_vector[5])

    return circuit

In [19]:
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2 ** n_qubits, 2 ** n_qubits))
projector[0, 0] = 1

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    encode_data(x1)
    qml.adjoint(encode_data)(x2)
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [20]:
kernel(X_train[0], X_train[0])

IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
def kernel_matrix(A, B):
    return np.array([[kernel(a, b) for b in B] for a in A])

In [ ]:
best_svc = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [ ]:
svc_pred = best_svc.predict(X_test)

# Evaluate the model
print(confusion_matrix(y_test, svc_pred))
print(classification_report(y_test, svc_pred,zero_division=True))

accuracy_score(y_test, svc_pred)

[[ 0 12]
 [ 0 87]]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00        12
           1       0.88      1.00      0.94        87

    accuracy                           0.88        99
   macro avg       0.94      0.50      0.47        99
weighted avg       0.89      0.88      0.82        99



0.8787878787878788

In [ ]:
def circuit_evals_kernel(n_data, split):
    """Compute how many circuit evaluations one needs for kernel-based
       training and prediction."""

    M = int(np.ceil(split * n_data))
    Mpred = n_data - M

    n_training = M * M
    n_prediction = M * Mpred

    return n_training + n_prediction

In [ ]:
circuit_evals_kernel(n_data=len(X), split=len(X_train) / (len(X_train) + len(X_test)))

60300